<a href="https://colab.research.google.com/github/jorcisai/APR/blob/master/neuralnets/src/mnist_local_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST loader

Importing standard packages and tensorflow_datasets to ease data manipulation.

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf
import os

import tensorflow_datasets as tfds

First, you need to upload mnist_dataset.py and original MNIST data files from your drive

In [ ]:
from google.colab import files

uploaded = files.upload()

Loading training and test images and labs from array into dataset object. Then, creating pairs of (image,label)

In [27]:
from mnist_dataset import get_mnist

(X, xl), (Y, yl) = get_mnist("./").load_data()
N = X.shape[0]
C = len(set(xl))
Xdat   = tf.data.Dataset.from_tensor_slices(X)
xldat  = tf.data.Dataset.from_tensor_slices(xl)
Xxldat = tf.data.Dataset.zip((Xdat, xldat))
Ydat   = tf.data.Dataset.from_tensor_slices(Y)
yldat  = tf.data.Dataset.from_tensor_slices(yl)
Yyldat = tf.data.Dataset.zip((Ydat, yldat))

Taking a look at a couple of training images and labels after being converted into dataset type.

In [ ]:
for (image,label) in Xxldat.take(1):
  print(label)
  print(image)

# Experimental design

Use tf.data.Dataset.take and tf.data.Dataset.skip to split training dataset into 90% for training and 10% for dev.

Before being passed into the model, the datasets need to be shuffled and batched. So, first, the complete dataset is shuffled with a fixed seed so that we can repeat the same shuffle of the dataset, then the dataset is split into training, validation and test, and each of these subsets is batched.

In [29]:
Ntr = round(N*0.9)
Ndv = N-Ntr
Nbatch = 100

Xxldat = Xxldat.shuffle(N,seed=13)

Xxltr = Xxldat.take(Ntr)
Xxltr = Xxltr.batch(Nbatch)
Xxldv = Xxldat.skip(Ntr).take(Ndv)
Xxldv = Xxldv.batch(Nbatch)
Yylte = Yyldat.batch(Nbatch)

# Build the model

Create an empty model and add layers to it.

In [30]:
model = tf.keras.Sequential()

We'll have a series of one or more densely connected layers, with the last one being the output layer. The output layer produces a probability for all the labels. The one with the highest probability is the models prediction of an example's label.

In [31]:
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [16, 16]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(C, activation='softmax'))

Finally, compile the model. For a softmax categorization model, use sparse_categorical_crossentropy as the loss function. You can try other optimizers, but adam is very common.

In [32]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model

This model running for 10 epochs on this data produces decent results

In [33]:
model.fit(Xxltr, epochs=10, validation_data=Xxldv)

Epoch 1/10
540/540 [==============================] - 1s 3ms/step - loss: 2.7265 - accuracy: 0.2250 - val_loss: 1.8890 - val_accuracy: 0.2767
Epoch 2/10
540/540 [==============================] - 1s 3ms/step - loss: 1.7473 - accuracy: 0.3382 - val_loss: 1.5579 - val_accuracy: 0.3938
Epoch 3/10
540/540 [==============================] - 1s 3ms/step - loss: 1.4422 - accuracy: 0.4189 - val_loss: 1.3572 - val_accuracy: 0.4520
Epoch 4/10
540/540 [==============================] - 1s 2ms/step - loss: 1.2520 - accuracy: 0.4767 - val_loss: 1.1797 - val_accuracy: 0.4955
Epoch 5/10
540/540 [==============================] - 1s 2ms/step - loss: 1.1495 - accuracy: 0.5227 - val_loss: 1.1140 - val_accuracy: 0.5483
Epoch 6/10
540/540 [==============================] - 1s 2ms/step - loss: 1.0611 - accuracy: 0.5745 - val_loss: 1.0181 - val_accuracy: 0.5812
Epoch 7/10
540/540 [==============================] - 1s 3ms/step - loss: 1.0059 - accuracy: 0.6010 - val_loss: 0.9569 - val_accuracy: 0.6163
Epoch 

In [3]:
ls

sample_data/
